# DRP Processing Flags

In [ ]:
# First lets get a source catalog to play with:
# source /global/common/cori/contrib/lsst/lsstDM/setupStack-13_0_c.sh
# setup lsst_distrib

import lsst.daf.persistence as dafPersist

butler = dafPersist.Butler('/global/cscratch1/sd/descdm/DC1/full_focalplane_undithered')
srcCat = butler.get('deepCoadd_meas', filter='r', tract=0, patch='7,9')
srcCat.getSchema()

More detail about the flags seen in this schema:

## 1) Primary Flag

The most important flag is the "detect_isPrimary" flag. Any kind analysis should be done only for sources "where detect_isPrimary".

There are two types of duplicates in the source catalogs. (a) Deblender parents vs. their children and (b) sources measured in the patch and tract overlap regions. "detect_isPrimary=True" will ensure that you have ONLY the children sources when a footprint was deblended and only sources in the inner patch and inner tract. 

    (Field['Flag'](name="detect_isPatchInner", doc="true if source is in the inner region of a coadd patch"), Key['Flag'](offset=544, bit=62)),
    (Field['Flag'](name="detect_isTractInner", doc="true if source is in the inner region of a coadd tract"), Key['Flag'](offset=544, bit=63)),
    (Field['Flag'](name="detect_isPrimary", doc="true if source has no children and is in the inner region of a coadd patch and is in the inner region of a coadd tract and is not "detected" in a pseudo-filter (see config.pseudoFilterList)"), Key['Flag'](offset=1168, bit=0)),
    
    
You can also only select children by limiting to "where deblend_nChild = 0"

    (Field['I'](name="deblend_nChild", doc="Number of children this object has (defaults to 0)"), Key<I>(offset=40, nElements=1)),
    
    
    detect_isPrimary = deblend_nChild == 0 and detect_isPatchInner and detect_isTractInner

 

In [ ]:
srcCat.get("deblend_nChild")
srcCat.get("detect_isPrimary")
srcCat.get("parent")

# Limit to primary like:
idx = srcCat.get("detect_isPrimary") 
print len(srcCat.get("base_PsfFlux_flux")[idx])

## 2) Pixel Flags

For clean samples, you may also want to filter sources w/ bad pixels. We often filter out "base_PixelFlags_flag_edge", "base_PixelFlags_flag_saturated", "base_PixelFlags_flag_saturatedCenter." Using "base_PixelFlags_flag" will filter out all of the flags thtat start with "base_PixelFlags_flag*".

    (Field['Flag'](name="base_PixelFlags_flag", doc="general failure flag, set if anything went wring"), Key['Flag'](offset=544, bit=22)),
    (Field['Flag'](name="base_PixelFlags_flag_offimage", doc="Source center is off image"), Key['Flag'](offset=544, bit=23)),
    (Field['Flag'](name="base_PixelFlags_flag_edge", doc="Source is outside usable exposure region (masked EDGE or NO_DATA)"), Key['Flag'](offset=544, bit=24)),
    (Field['Flag'](name="base_PixelFlags_flag_interpolated", doc="Interpolated pixel in the Source footprint"), Key['Flag'](offset=544, bit=25)),
    (Field['Flag'](name="base_PixelFlags_flag_saturated", doc="Saturated pixel in the Source footprint"), Key['Flag'](offset=544, bit=26)),
    (Field['Flag'](name="base_PixelFlags_flag_cr", doc="Cosmic ray in the Source footprint"), Key['Flag'](offset=544, bit=27)),
    (Field['Flag'](name="base_PixelFlags_flag_bad", doc="Bad pixel in the Source footprint"), Key['Flag'](offset=544, bit=28)),
    (Field['Flag'](name="base_PixelFlags_flag_suspect", doc="Source''s footprint includes suspect pixels"), Key['Flag'](offset=544, bit=29)),
    (Field['Flag'](name="base_PixelFlags_flag_interpolatedCenter", doc="Interpolated pixel in the Source center"), Key['Flag'](offset=544, bit=30)),
    (Field['Flag'](name="base_PixelFlags_flag_saturatedCenter", doc="Saturated pixel in the Source center"), Key['Flag'](offset=544, bit=31)),
    (Field['Flag'](name="base_PixelFlags_flag_crCenter", doc="Cosmic ray in the Source center"), Key['Flag'](offset=544, bit=32)),
    (Field['Flag'](name="base_PixelFlags_flag_suspectCenter", doc="Source''s center is close to suspect pixels"), Key['Flag'](offset=544, bit=33)),

## 3) Measurement Flags

Measurement algorithms have their own flags to record exceptions and abnormalities detected in the measurement. Every measurement algorithm will have a unique series of flags to record the unique failure modes and a general flag which is the "or" of the individual flags. For example:

    (Field['D'](name="base_PsfFlux_flux", doc="flux derived from linear least-squares fit of PSF model", units="count"), Key<D>(offset=696, nElements=1)),
    (Field['D'](name="base_PsfFlux_fluxSigma", doc="1-sigma flux uncertainty", units="count"), Key<D>(offset=704, nElements=1)),
    (Field['Flag'](name="base_PsfFlux_flag", doc="general failure flag"), Key['Flag'](offset=544, bit=34)),
    (Field['Flag'](name="base_PsfFlux_flag_noGoodPixels", doc="not enough non-rejected pixels in data to attempt the fit"), Key['Flag'](offset=544, bit=35)),
    (Field['Flag'](name="base_PsfFlux_flag_edge", doc="object was too close to the edge of the image to use the full PSF model"), Key['Flag'](offset=544, bit=36)),


base_PsfFlux_flag = base_PsfFlux_flag_noGoodPixels or base_PsfFlux_flag_edge

In [ ]:
srcCat.get("base_PsfFlux_flag")

In [ ]:
# NOT a DM-endorsed way of working with tables:

import pandas as pd

df = pd.DataFrame()
for col in srcCat.getSchema():
    name = col.field.getName()
    df[name] = icsrc.get(name)

df.to_csv('filename.csv')